# LOGISTIC REGRESSION MODEL

## DATA PREPROCESSING.

###  install packages on the python environment, if not yet present

In [34]:
# %pip install -I jinja2==3.0.3 --user
# %pip install scikit-learn==1.1.3
# %pip install statsmodels
# %pip install particles
# %pip install pandas
# %pip install matplotlib
# %conda install -c "conda-forge/label/broken" nbconvert-webpdf
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.feature_selection import VarianceThreshold
%matplotlib inline
import particles
from particles import distributions as dists
from particles import resampling as rs
from particles import smc_samplers as ssps
from particles.collectors import Moments
from sklearn.metrics import confusion_matrix
import seaborn as sns

### ETS steps on KDD99 set with feature reduction

#### Import,variance treshold, multicollinearity

In [35]:
# import the dataset as a pandas dataframe
train_data_url = "https://raw.githubusercontent.com/stijnhuysman/DissertationData/main/Train_data.csv"
dftot = pd.read_csv(train_data_url)

print("obs and features :",dftot.shape)
####################
# FEATURES DATASET #
####################
# creating dummy variables on the categorical values
Xtrain = pd.get_dummies(data=dftot.iloc[:,:(-1)] ,  
                        drop_first=True,
                        columns=['protocol_type', 'service', 'flag']
                         , dtype=np.uint0)
print("obs full dum", Xtrain.shape)
def variance_threshold_selector(data, threshold=0.5):
    selector = VarianceThreshold(threshold*(1-threshold))
    selector.fit_transform(data)
    return data[data.columns[selector.get_support(indices=True)]]

#reduced dataframe where 80 %  variance is explained
Xtrain_selected = variance_threshold_selector(Xtrain, 0.8)
print("amount of features in the training dataset after variance reductions: " ,len(Xtrain_selected.columns))
# Xtrain_selected.head()


####################
# RESPONSE VARIABLES #
####################

Y = dftot.loc[:,'class']
Y = pd.get_dummies(data=Y, drop_first = True)


####################
# MERGING DATA      #
####################

Intrusion_full = pd.merge(Xtrain_selected, Y, left_index = True, right_index=True)
# Intrusion_full.count()

####################
# MULTICOLLINEARITY REDUCTION      #
####################
# Use variance inflation factor to identify any significant multi-collinearity
# !!!!!! ONLY THE FEATURES WITH VIF less or equal to 5 are kept !!!!!
# 5 as setpoint #


from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(df):
    vif = pd.DataFrame()
    vif["variables"] = df.columns
    vif["VIF"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    return(vif)

VIF_table = calc_vif(Xtrain_selected).sort_values(by=['VIF'], ascending=False)
VIF_select = VIF_table[(VIF_table['VIF'] <= 5)]




selection = VIF_select.iloc[:,0].values.tolist()[-5:]
selection = VIF_select.iloc[:,0].values.tolist()
X_selected = Xtrain_selected[selection]

print("amount of features in the training dataset after multicollinearity reductions: " ,len(X_selected.columns))
print("amount of observations in the training dataset before subsetting: " ,len(X_selected.index))
Intrusion = pd.merge(X_selected, Y, left_index = True, right_index=True)
Intrusion.head()
# column_names = keeping a list of the columnames for later use
column_names = Intrusion.columns.tolist()[:-1]  

column_names
Intrusion.head()
# print(selection, column_names)
column_names


obs and features : (25192, 42)
obs full dum (25192, 115)
amount of features in the training dataset after variance reductions:  21
amount of features in the training dataset after multicollinearity reductions:  7
amount of observations in the training dataset before subsetting:  25192


['count',
 'srv_count',
 'duration',
 'hot',
 'num_file_creations',
 'dst_bytes',
 'src_bytes']

#### Standardize selected features for logistic regression

In [36]:
####################
# STANDARDIZE DATA FOR LOGISTIC REGRESSION      #
####################
# here following the procedure as explained by Chopin : outcome values as -1 and 1
# also the features are normalized to each have their normal(0,1) distribution
# finally, from here, all data is handled as arrays, for speeding up calculations

def prepare_predictors(predictors, add_intercept=True, scale=0.5):
    preds = np.atleast_2d(predictors)  # in case predictors is (n,)
    rescaled_preds = scale * (preds - np.mean(preds, axis=0)) / np.std(preds, axis=0)
    if add_intercept:
        n, p = preds.shape
        out = np.empty((n, p + 1))
        out[:, 0] = 1. # intercept
        out[:, 1:] = rescaled_preds
    else:
        out = rescaled_preds
    return out

   
    
def preprocess(raw_data, return_y=False):
    response = 2 * raw_data[:, -1] - 1  # 0/1 -> -1/1
    preds = prepare_predictors(raw_data[:, :-1])
    if return_y:
        return preds, response
    else:
        return preds * response[:, np.newaxis]
    
    
raw_data = Intrusion.to_numpy()
data = preprocess(raw_data)

T, p = data.shape  
print("The data has ", T, " observations and ",p-1," features." )
data[:10]
T

The data has  25192  observations and  7  features.


25192

#### Defining outcome array  Y and Selected features X, including a Unit (1) column

In [37]:

# Y = data[:, [0]]
# # X = data[:, [1,8]]
# # X = np.delete(data, 0, axis=1)
# X = data
# X[:,0] = np.ones(T)  
# X[:3]   #for the X calc, the first column is replaced by 1, 
# # in a later step, this will be used for matrix calculation of the intercept
# Y

#### Subsetting data

In [38]:
#SUBSET DATA TO nobs  OBSERVATIONS in line
import numpy as np
observations = T
nobs = T
# nobs = 15000  #HERE CHOOSING A SPECIFIC AMOUNT
startpos = int(np.random.randint(low = 0, high = observations - nobs+1, size=1,dtype=np.int64))
# startpos = 0
endpos = startpos + nobs
# Intrusion = Intrusion_full.iloc[startpos:endpos]
# print('Subsetted the data to ', len(Intrusion.index), ' sequential observations.')


# the dataset data is a list of arrays
dataset = data[startpos:endpos]
len(dataset)
T, p = dataset.shape  
print("The data has ", T, " observations and ",p-1," features." )
# print(dataset)

Y =  [int(elem[0]) for elem in dataset]
# print(Y)

X = dataset.copy()
X[:,0] = np.ones(T)  
#for the X calc, the first column is replaced by 1, 
# in a later step, this will be used for matrix calculation of the intercept


The data has  25192  observations and  7  features.


## LOGISTIC REGRESSION MODEL  and FK objects

### Setting parameters

In [39]:
######ENTER_PARAMETER#######
alg_type = 'ibis'
N_part = [10, 100, 1000]                          #  N0  = 200 000 => number of resampled particles
p  #amount of betas

print("amount of betas: ", p)
print("different amount of particles sampled: " ,N_part) 



amount of betas:  8
different amount of particles sampled:  [10, 100, 1000]


### Defining the Bayesian model for logistic regression

In [40]:
# PRIOR AND MODEL
### DEFINITION OF THE PRIOR PARAMETERS
######ENTER_PARAMETER#######

scales = 0.8 * np.ones(p)     # for all column beta scales  of 5 = >variance
scales[0] = 20  # intercept has a larger scale   #the beta_0 of course is larger, we adjust it to start from

### AMOUNT OF RUNS OF THE SAMPLER
nruns = 1

###empty result list to track the summaries
results = []


######################################################
# #################PRIOR###########################
######################################################

'''Define a prior with multivariate Normal distro'''

prior = dists.StructDist({'beta':dists.MvNormal(scale=scales,
                                                cov=np.eye(p))})

#the prior is nothing more than a MVN distribution with an eye cov matrix

######################################################
# ###BAYESIAN MODEL : LIKELIHOOD X PRIOR #############
######################################################

class LogisticRegression(ssps.StaticModel):  #build  a SMC sampler LR class
    def logpyt(self, theta, t):
        # log-likelihood factor t, for given theta
        lin = np.matmul(theta['beta'], data[t, :])
        #Matrix product of two arrays.  => the logmodil takes all X-es and multiplies with the beta matrix (that is a multivariate normal of dimention
        
        
        return - np.logaddexp(0., -lin)

    #  the numpy.logaddexp() function is used to calculate Logarithm of the sum of exponentiations of the inputs.
    #  This function is useful in statistics where the calculated probabilities of events 
    # may be so small as to exceed the range of normal floating point numbers.  In such cases
    # the logarithm of the calculated probability is stored. This function
    # allows adding probabilities stored in such a fashion.
    
    
# nruns = 100
# nsteps = 100
prior.dtype  

[('beta', 'float64', 8)]

'''
now we model the Logistic Regression as a classifier and build a sampler for it

interesting info :  see also : https://towardsdatascience.com/bayesian-logistic-regression-53df017ba90f

The starting point for Bayesian Logistic Regression is Bayes’ Theorem, 
which formally states that the posterior distribution of parameters 
is proportional to the product of two quantities: 
the likelihood of observing the data given the parameters and the prior density of parameters. 
Applied to our context this can intuitively be understood as follows: 

our posterior beliefs around the logistic regression coefficients are formed by 
both our prior beliefs and the evidence we observe (i.e. the data).

Under the assumption that individual label-feature pairs are independently and identically distributed,
their joint likelihood is simply the product over their individual densities (Bernoulli).

Solving the problem
In practice we do not maximize the posterior directly. 
Instead we minimize the negative log likelihood, which is equivalent and easier to implement.

'''


### Create a FK object based on the model

### Define the options to resample

In [41]:
#################################"""
#RESAMPLING METHODS

# print('Dataset: %s' % dataset_name)
# resampling = ['multinomial', 'residual', 'stratified', 'systematic', 'ssp']
# for rsmethod in resampling:
    # print(rsmethod)

    
######ENTER_PARAMETER#######
resampling = ['systematic', 'ssp','stratified']
# resampling = 'systematic'


## RUN SAMPLERS

### Run multiple particle filters, using multiple processors together

  !!!***CAREFUL THE CELL CAN TAKE A LOT OF TIME BASED ON COMBINATIONS AND PARAMS*** !!!
  current params :
  The time of execution of above program with  3  combinatons took : 780.7906959056854 s

PARAMETER BOX


In [42]:
N_part = list(range(10,501,20))
len(N_part)
chains = list(range(10, 101, 20))
len(chains)

5

In [43]:
nruns =1
# N_part = [10, 1000, 5000]
# particles.multiSMC?
scales = 0.5 * np.ones(p)     
scales[0] = 20  
prior = dists.StructDist({'beta':dists.MvNormal(scale=scales, cov=np.eye(p))})
model = LogisticRegression(data=dataset[-5000:], prior=prior) #the model is the base for the FK element 
fullmodel = LogisticRegression(data=dataset[:], prior=prior) #the model is the base for the FK element 

fk10 = ssps.IBIS(model=model, len_chain=10)
fk20 = ssps.IBIS(model=model, len_chain=20)
fk30 = ssps.IBIS(model=model, len_chain=30)

fk_objects = [fk10, fk20, fk30]


In [44]:
import time
import particles

## THIS SAMPLER BELOW CALCULATES ALL THE POSSIBILITIES

In [45]:
# Import time module
import time
import particles
from particles import smc_samplers as ssps

duration_part = []
MASTER = []
# for c in chains:
for c in [10, 50]:
    fk = ssps.IBIS(model=fullmodel, len_chain=c)
    for n in [100,300]:
        start = time.time()
        resultaat = particles.multiSMC(nruns=nruns,     #A sampling runs
                                       nprocs=0,
                                       fk=fk,   #B FK objects
                                       N=n,           #C N options 
                                       collect=[Moments], 
                                       verbose=False,
                                       resampling = 'systematic')  #D  resampling options
        # resultaat = particles.multiSMC(nruns=1, fk=fk, N=50, collect=[Moments], verbose=False, resampling = 'systematic')

        #AANTAL COMBINATIES = A*B*C*D
        # combinations = nruns*len(fk_objects)*len(N_part)*1
        combinations = len(resultaat)    
        MASTER.append(resultaat)
        # # endtime and print
        end = time.time()
        duration_part.append(end-start)
        print("The time of execution of above program with ", combinations, " combinatons and ", n, " particles and chain: ", c, "took :",
              (end-start) , "s")

duration_part


The time of execution of above program with  1  combinatons and  100  particles and chain:  10 took : 89.87825179100037 s
The time of execution of above program with  1  combinatons and  300  particles and chain:  10 took : 179.435485124588 s
The time of execution of above program with  1  combinatons and  100  particles and chain:  50 took : 392.65454864501953 s
The time of execution of above program with  1  combinatons and  300  particles and chain:  50 took : 724.4963138103485 s


[89.87825179100037, 179.435485124588, 392.65454864501953, 724.4963138103485]

In [46]:
## SAME CALCULATEION BUT MULTIPROCESSED

In [47]:
# Import time module
import time
import particles
from particles import smc_samplers as ssps

fk = [ssps.IBIS(model=fullmodel, len_chain=10), ssps.IBIS(model=fullmodel, len_chain=50)]
start = time.time()
MASTER2 = particles.multiSMC(nruns=nruns,     #A sampling runs
                               nprocs=0,
                               fk=fk,   #B FK objects
                               N=[100,300],           #C N options 
                               collect=[Moments], 
                               verbose=False,
                               resampling = 'systematic')  #D  resampling options
end = time.time()
duration_part.append(end-start)
print("The time of execution of above program with ", len(MASTER2), " combinations, " , "took :",
      (end-start) , "s")




The time of execution of above program with  4  combinations,  took : 744.6186916828156 s


In [48]:
MASTER2

[{'run': 0,
  'fk': <particles.smc_samplers.IBIS at 0x251ceb29700>,
  'N': 100,
  'seed': 434510515,
  'output': <particles.core.SMC at 0x251ceb02880>},
 {'run': 0,
  'fk': <particles.smc_samplers.IBIS at 0x251ceb29700>,
  'N': 300,
  'seed': 1389608321,
  'output': <particles.core.SMC at 0x251ceb0ffd0>},
 {'run': 0,
  'fk': <particles.smc_samplers.IBIS at 0x251ceb29310>,
  'N': 100,
  'seed': 2234645933,
  'output': <particles.core.SMC at 0x2519138d430>},
 {'run': 0,
  'fk': <particles.smc_samplers.IBIS at 0x251ceb29310>,
  'N': 300,
  'seed': 3541807894,
  'output': <particles.core.SMC at 0x251ceb02c70>}]

In [49]:
# chain, N, duration = null
TRACKING = []
for i in range(len(MASTER)):
# for i in [0]:

    res = MASTER[i][0]
    
    TRACKING.append((MASTER[i][0]['output'].N, MASTER[i][0]['output'].fk.len_chain, MASTER[i][0]['output'].cpu_time))
    
TRACKING

[(100, 10, 84.307419),
 (300, 10, 175.5365938),
 (100, 50, 389.30648750000006),
 (300, 50, 720.9583277)]

In [50]:
# Import time module


duration_part = []
MASTER = []

#DO NOT EXECUTE!!!! TAKES TIME
for fk in fk_objects:
    for n in N_part:
        start = time.time()
        resultaat = particles.multiSMC(nruns=nruns,     #A sampling runs
                                       nprocs=0,
                                       fk=fk,   #B FK objects
                                       N=n,           #C N options 
                                       collect=[Moments], 
                                       verbose=False,
                                       resampling = 'systematic')  #D  resampling options
        # resultaat = particles.multiSMC(nruns=1, fk=fk, N=50, collect=[Moments], verbose=False, resampling = 'systematic')

        #AANTAL COMBINATIES = A*B*C*D
        # combinations = nruns*len(fk_objects)*len(N_part)*1
        combinations = len(resultaat)    
        MASTER.append(resultaat)
        # # endtime and print
        end = time.time()
        duration_part.append(end-start)
        print("The time of execution of above program with ", combinations, " combinatons and ", n, " particles took :",
              (end-start) , "s")

duration_part


LinAlgError: 8-th leading minor of the array is not positive definite

In [ ]:
print("Combinations: ",(MASTER[0][0]['output'].N, MASTER[1][0]['output'].fk.len_chain),
     (MASTER[1][0]['output'].N, MASTER[1][0]['output'].fk.len_chain),
     (MASTER[2][0]['output'].N, MASTER[1][0]['output'].fk.len_chain),
     (MASTER[3][0]['output'].N, MASTER[1][0]['output'].fk.len_chain),)

      

In [ ]:

MASTER[1][0]['output'].summaries.moments[:2]

## RESULTS

In [ ]:
# CALCULATE ALL POSSIBILITIES
a = T   #if we use T : we use ALL the observations on scope

a = 0# for i in range(len(resultaat)):   #for EVERY COMBINATION

SUMMARY = []
for i in range(len(MASTER2)):   #for EVERY COMBINATION
    OUT = MASTER2[i]['output']
    MOMENTS = OUT.summaries.moments[a:]
    particles_used =OUT.N
    chainlength = OUT.fk.len_chain
    betameans_set = [entry['mean'].tolist()[0].tolist() for entry in MOMENTS]
    betavar_set = [entry['var'].tolist()[0].tolist() for entry in MOMENTS ]
    upper_set = [entry['mean'].tolist()[0].tolist() + 1.96*np.sqrt(entry['var'].tolist()[0].tolist()) for entry in MOMENTS]
    lower_set = [entry['mean'].tolist()[0].tolist() - 1.96*np.sqrt(entry['var'].tolist()[0].tolist()) for entry in MOMENTS]
    
     # Listing the estimates per feature (beta, var, under , upper), ready to plot
    SUMMARY.append((particles_used, chainlength, betameans_set, betavar_set, lower_set,upper_set) )
    
print(SUMMARY[0][0], SUMMARY[0][1])
print(SUMMARY[1][0], SUMMARY[1][1])
print(SUMMARY[2][0], SUMMARY[2][1])
print(SUMMARY[3][0], SUMMARY[3][1])


amountofbetas = len(SUMMARY[0][2][0])

for i in range(len(SUMMARY)):
    #############################################################
    #NOW TAKE ALL VALUES FOR THE BETAS AND CONFIDENCE BANDS
    #############################################################
    
    for j in range(amountofbetas):
        globals()[f"beta{i}_{j}"] = [sublist[j] for sublist in SUMMARY[i][2]]
        globals()[f"var{i}_{j}"] = [sublist[j] for sublist in SUMMARY[i][3]]
        globals()[f"under{i}_{j}"] = [sublist[j] for sublist in SUMMARY[i][4]]
        globals()[f"upper{i}_{j}"] = [sublist[j] for sublist in SUMMARY[i][5]]

    # Listing a numbered list of observations
    observations = list(range(len(beta0_0)))
    particles_used 


### PLOTTING

In [ ]:
#############################################################
#PLOT THEM 
#############################################################

fig, axs = plt.subplots(4, 2, figsize=(30, 40))
fig.suptitle(
    str('Logistic regression')).set_fontsize(60)

fig.subplots_adjust(bottom=.60)
ft = 40

linewidth = 6
col = 'lightgrey'
col = "#EBECF0"


axs[0,0].plot(observations, beta0_0 , label='N: 100, chain 10 ', linewidth=linewidth)
axs[0,0].plot(observations, beta1_0 , label='N: 300, chain 10 ', linewidth=linewidth)
axs[0,0].plot(observations, beta2_0 , label='N: 100, chain 50 ', linewidth=linewidth)
axs[0,0].plot(observations, beta3_0, label='N: 300, chain 50 ', linewidth=linewidth)
axs[0,0].legend(fontsize=ft)
axs[0,0].set_facecolor(col)



# axs[0,0].fill_between(observations, under0_0, upper0_0, color='gray', alpha=0.5)
# axs[0,0].fill_between(observations, under1_0, upper1_0, color='gray', alpha=0.5)
# axs[0,0].fill_between(observations, under2_0, upper2_0, color='gray', alpha=0.5)
# axs[0,0].fill_between(observations, under3_0, upper3_0, color='gray', alpha=0.5)

axs[0,1].plot(observations, beta0_1, label='N: 100, chain 10 ', linewidth=linewidth)
axs[0,1].plot(observations, beta1_1, label='N: 300, chain 10 ', linewidth=linewidth)
axs[0,1].plot(observations, beta2_1, label='N: 100, chain 50 ', linewidth=linewidth)
axs[0,1].plot(observations, beta3_1, label='N: 300, chain 50 ', linewidth=linewidth)
axs[0,1].legend(fontsize=ft)
axs[0,1].set_facecolor(col)



axs[1,0].plot(observations, beta0_2, label='N: 100, chain 10 ', linewidth=linewidth)
axs[1,0].plot(observations, beta1_2, label='N: 300, chain 10 ', linewidth=linewidth)
axs[1,0].plot(observations, beta2_2, label='N: 100, chain 50 ', linewidth=linewidth)
axs[1,0].plot(observations, beta3_2, label='N: 300, chain 50 ', linewidth=linewidth)
axs[1,0].legend(fontsize=ft)
axs[1,0].set_facecolor(col)

axs[1,1].plot(observations, beta0_3, label='N: 100, chain 10 ', linewidth=linewidth)
axs[1,1].plot(observations, beta1_3, label='N: 300, chain 10 ', linewidth=linewidth)
axs[1,1].plot(observations, beta2_3, label='N: 100, chain 50 ', linewidth=linewidth)
axs[1,1].plot(observations, beta3_3, label='N: 300, chain 50 ', linewidth=linewidth)
axs[1,1].legend(fontsize=ft)
axs[1,1].set_facecolor(col)

axs[2,0].plot(observations, beta0_4, label='N: 100, chain 10 ', linewidth=linewidth)
axs[2,0].plot(observations, beta1_4, label='N: 300, chain 10 ', linewidth=linewidth)
axs[2,0].plot(observations, beta2_4, label='N: 100, chain 50 ', linewidth=linewidth)
axs[2,0].plot(observations, beta3_4, label='N: 300, chain 50 ', linewidth=linewidth)
axs[2,0].legend(fontsize=ft)
axs[2,0].set_facecolor(col)

axs[2,1].plot(observations, beta0_5, label='N: 100, chain 10 ', linewidth=linewidth)
axs[2,1].plot(observations, beta1_5, label='N: 300, chain 10 ', linewidth=linewidth)
axs[2,1].plot(observations, beta2_5, label='N: 100, chain 50 ', linewidth=linewidth)
axs[2,1].plot(observations, beta3_5, label='N: 300, chain 50 ', linewidth=linewidth)
axs[2,1].legend(fontsize=ft)
axs[2,1].set_facecolor(col)

axs[3,0].plot(observations, beta0_6, label='N: 100, chain 10 ', linewidth=linewidth)
axs[3,0].plot(observations, beta1_6, label='N: 300, chain 10 ', linewidth=linewidth)
axs[3,0].plot(observations, beta2_6, label='N: 100, chain 50 ', linewidth=linewidth)
axs[3,0].plot(observations, beta3_6, label='N: 300, chain 50 ', linewidth=linewidth)
axs[3,0].legend(fontsize=ft)
axs[3,0].set_facecolor(col)

axs[3,1].plot(observations, beta0_7, label='N: 100, chain 10 ', linewidth=linewidth)
axs[3,1].plot(observations, beta1_7, label='N: 300, chain 10 ', linewidth=linewidth)
axs[3,1].plot(observations, beta2_7, label='N: 100, chain 50 ', linewidth=linewidth)
axs[3,1].plot(observations, beta3_7, label='N: 300, chain 50 ', linewidth=linewidth)
axs[3,1].legend(fontsize=ft)
axs[3,1].set_facecolor(col)


m = .05


under0 = np.min([beta0_0,beta1_0, beta2_0, beta3_0])
upper0 = np.max([beta0_0,beta1_0, beta2_0, beta3_0])
under1 = np.min([beta0_1,beta1_1, beta2_1, beta3_1])
upper1 = np.max([beta0_1,beta1_1, beta2_1, beta3_1])
under2 = np.min([beta0_2,beta1_2, beta2_2, beta3_2])
upper2 = np.max([beta0_2,beta1_2, beta2_2, beta3_2])
under3 = np.min([beta0_3,beta1_3, beta2_3, beta3_3])
upper3 = np.max([beta0_3,beta1_3, beta2_3, beta3_3])
under4 = np.min([beta0_4,beta1_4, beta2_4, beta3_4])
upper4 = np.max([beta0_4,beta1_4, beta2_4, beta3_4])
under5 = np.min([beta0_5,beta1_5, beta2_5, beta3_5])
upper5 = np.max([beta0_5,beta1_5, beta2_5, beta3_5])
under6 = np.min([beta0_6,beta1_6, beta2_6, beta3_6])
upper6 = np.max([beta0_6,beta1_6, beta2_6, beta3_6])
under7 = np.min([beta0_7,beta1_7, beta2_7, beta3_7])
upper7 = np.max([beta0_7,beta1_7, beta2_7, beta3_7])

#     #set the y limits manually
axs[0,0].set_ylim([np.min(under0)-np.abs(np.min(under0))*m,np.max(upper0 + np.abs(np.max(upper0))*m)])
axs[0,1].set_ylim([np.min(under1)-np.abs(np.min(under1))*m,np.max(upper1 + np.abs(np.max(upper1))*m)])
axs[1,0].set_ylim([np.min(under2)-np.abs(np.min(under2))*m,np.max(upper2 + np.abs(np.max(upper2))*m)])
axs[1,1].set_ylim([np.min(under3)-np.abs(np.min(under3))*m,np.max(upper3 + np.abs(np.max(upper3))*m)])
axs[2,0].set_ylim([np.min(under4)-np.abs(np.min(under4))*m,np.max(upper4 + np.abs(np.max(upper4))*m)])
axs[2,1].set_ylim([np.min(under5)-np.abs(np.min(under5))*m,np.max(upper5 + np.abs(np.max(upper5))*m)])
axs[3,0].set_ylim([np.min(under6)-np.abs(np.min(under6))*m,np.max(upper6 + np.abs(np.max(upper6))*m)])
axs[3,1].set_ylim([np.min(under7)-np.abs(np.min(under7))*m,np.max(upper7 + np.abs(np.max(upper7))*m)])
#   

# set the title to subplots
axs[0, 0].set_title('intercept').set_fontsize(ft)
axs[0, 1].set_title(column_names[0]).set_fontsize(ft)
axs[1, 0].set_title(column_names[1]).set_fontsize(ft)
axs[1, 1].set_title(column_names[2]).set_fontsize(ft)
axs[2, 0].set_title(column_names[3]).set_fontsize(ft)
axs[2, 1].set_title(column_names[4]).set_fontsize(ft)
axs[3, 0].set_title(column_names[5]).set_fontsize(ft)
axs[3, 1].set_title(column_names[6]).set_fontsize(ft)




# set spacing
fig.tight_layout()

plt.savefig('logistic_regression2', dpi = 300)
plt.show()


opmerking = hier is een zekere drifting te zien van de parameters
het omslagpunt vanaf  meting 7000 valt op.

## CHECKING THE MODEL

### Logistic model input parameters

P(y=1|y) = 1/1+e-(b0 + b1x1 + .... + bnXn)

In [ ]:
i = 0
betameans_set = SUMMARY[i][2]
n = 4
varfeature = [sublist[n] for sublist in X]


In [ ]:
param = 0.5

a= 0
i = 3
betameans_set = SUMMARY[i][2]

##TRUE VALUES
Y_orig = [0 if y == -1 else 1 for y in Y][a:]

## LOGISTIC PREDICTION 

regressionterm_components = np.array(betameans_set[a:])*X[a:]
regressionterm = [np.sum(comp) for comp in regressionterm_components]
Y_pred = [1 / (1 + np.exp(-rt)) for rt in regressionterm] 
Y_est = [1 if y > param else 0 for y in Y_pred]
conf_mat = confusion_matrix(Y_orig, Y_est)

TP , FP, FN , TN = conf_mat.ravel()
Accuracy = (TP + TN) / (TP + TN + FP + FN)
Precision = TP / (TP + FP) 


# create labels
labels = ['INTRUSION', 'OK']

# create heatmap with labels
ax = sns.heatmap(conf_mat, annot=True, cmap='Blues', fmt='g', xticklabels=labels, yticklabels=labels)
ax.set_xlabel('Predicted')
ax.set_ylabel('True')

ax.set_title('Confusion Matrix\nAccuracy={:.2f}%, Precision={:.2f}%'.format(Accuracy*100, Precision*100))
plt.show()

In [ ]:
a = 0
i = 3
betameans_set = SUMMARY[i][2]

# TRUE VALUES
Y_orig = [0 if y == -1 else 1 for y in Y][a:]

# create labels
labels = ['INTRUSION', 'OK']

# plot two heatmaps horizontally
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

for j, param in enumerate([0.5,  0.7699999999999997]):
    # LOGISTIC PREDICTION
    regressionterm_components = np.array(betameans_set[a:]) * X[a:]
    regressionterm = [np.sum(comp) for comp in regressionterm_components]
    Y_pred = [1 / (1 + np.exp(-rt)) for rt in regressionterm]
    Y_est = [1 if y > param else 0 for y in Y_pred]
    conf_mat = confusion_matrix(Y_orig, Y_est)

    TP, FP, FN, TN = conf_mat.ravel()
    Accuracy = (TP + TN) / (TP + TN + FP + FN)
    Precision = TP / (TP + FP)

    # create heatmap with labels
    ax = sns.heatmap(conf_mat, annot=True, cmap='RdYlBu', fmt='g', xticklabels=labels, yticklabels=labels, ax=axs[j])
    ax.set_xlabel('Predicted')
    ax.set_ylabel('True')
    ax.set_title('Treshold = {:.2f}\nAccuracy = {:.2f}%, Precision = {:.2f}%'.format(param, Accuracy * 100, Precision * 100))
fig.suptitle('Confusion Matrix with default and tuned treshold ', fontsize=16, y=1.01)

plt.tight_layout()
plt.tight_layout()
plt.savefig('confusion logistic', dpi = 300)
plt.show()

In [ ]:
# Select the nth column
n = 4
nth_column = column_names[n]
print("Nth column:", nth_column)

# Create a vector with all columns except the nth
other_columns = [col for i, col in enumerate(column_names) if i != n]
print("Other columns:", other_columns)

In [ ]:
a = 0
import numpy as np
from sklearn.metrics import f1_score

# Define a range of parameter values to test
param_range = np.arange(0.1, 1.0, 0.01)

# Initialize variables to store the best parameter value and performance
best_param = 0
best_score = 0
best_accuracy = 0

# Loop through each parameter value and calculate the F1 score
for param in param_range:
    # Generate predictions based on the current parameter value
    Y_est = [1 if y > param else 0 for y in Y_pred]
    
    # Calculate the confusion matrix
    
    
    TP , FP, FN , TN = confusion_matrix(Y_orig[a:], Y_est[a:]).ravel()

    Accuracy = (TP + TN) / (TP + TN + FP + FN)
    Precision = TP / (TP + FP)  #correctly predicted divided by 
    
    
    # Calculate the F1 score based on the confusion matrix
    f1 = f1_score(Y_orig, Y_est)
    
    # If the current F1 score is better than the previous best and true negatives are at least 80%,
    # update the best values
    if Accuracy > best_accuracy and Precision >= .50:
        best_param = param
        best_accuracy = Accuracy
        best_precision = Precision

# Print the best parameter value and F1 score
print(f"Best parameter value: {best_param}")
print(f"precision: {best_precision}")
print(f"Accuracy: {Accuracy}")
